In [4]:
import pandas as pd

In [ ]:
# Load processed reviews
df = pd.read_csv("data/thematic_reviews.csv")

In [ ]:
import cx_Oracle
# Oracle connection (adjust with your credentials)
conn = cx_Oracle.connect(user="bank_reviews", password="strong_password", dsn="localhost/XE")
cursor = conn.cursor()

# Get unique banks and insert them
banks = df["bank"].unique()
bank_id_map = {}

for bank in banks:
    cursor.execute("INSERT INTO banks (name) VALUES (:1) RETURNING bank_id INTO :2", (bank, cursor.var(cx_Oracle.NUMBER)))
    bank_id = cursor.fetchone()[0]
    bank_id_map[bank] = bank_id

# Insert reviews
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO reviews (bank_id, review_text, rating, review_date, sentiment_label, sentiment_score, themes)
        VALUES (:1, :2, :3, TO_DATE(:4, 'YYYY-MM-DD'), :5, :6, :7)
    """, (
        bank_id_map[row["bank"]],
        row["review"],
        int(row["rating"]),
        row["date"][:10],
        row["sentiment_label"],
        float(row["sentiment_score"]),
        row["themes"]
    ))

conn.commit()
cursor.close()
conn.close()
print("✅ Data inserted into Oracle.")